In [ ]:
# -----------------------------导入python库------------------------------------
import os
import netCDF4 as nc
import pandas as pd
import numpy as np
import numpy.ma as ma
import xarray as xr
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.style as mplstyle
import cartopy.crs as ccrs
import netCDF4 as nc
from netCDF4 import Dataset
import shapefile
import time
import sys
# 将目标模块所在的路径添加到 sys.path 中
sys.path.append(r"/home/yfdong/data/work/LF-SAM/code/Library")
from MeteoVarPlot import draw_2DVAR, draw_Bias, draw_VARtemporal, draw_Bias_subplot,draw_2DVAR_subplot ,draw_2DuvVAR ,draw_uvBias_subplot ,draw_2DuvVAR_subplot
from MeteoChartPlot import add_TickGrid
from PreprocessVar import (
    get_MONTH_abbr, get_pentad, get_letter,
    preprocess_var, read_var,Events_configs,
    Read_VarsFromVarsDict, experiments_types,return_Event_Date,
    load_geodata, SubsetDomainConfigure ,create_domain_mask,get_letter,
    get_CoordPair
)
# 变量名转换表
var_dict = {
    
}
var_list = ['Prec', 'ET', 'LH', 'SH', 'SSM', 'SST', 'Q_2m', 'T_2m', 'PSFC', 'Wind_10m']
var_list = ['Prec', 'ET', 'SSM',  'Q_2m', 'T_2m', 'PSFC', 'Wind_10m']
sim_folder_mapping = {
    'ET': 'ET',
    'LH': 'LH',
    'SH': 'SH',
    'SSM': 'SH2O_Layer1',  # 土壤湿度第一层
    'RSM': 'SH2O_Layer3',  # 土壤湿度第三层
    # 'SST': 'TSLB_Layer1',  # 土壤温度第一层
    'Q_2m': 'Q2',
    'T_2m': 'T2',
    'PSFC': 'PSFC',
    'Wind_10m': 'Wind10',
    'Prec':'RAINNC',
}
sim_var_mapping = {
    'ET': 'ET',
    'LH': 'LH',
    'SH': 'SH',
    'SSM': 'SH2O_Layer1',  # 土壤湿度第一层
    'RSM': 'SH2O_Layer3',  # 土壤湿度第一层
    # 'SST': 'TSLB_Layer1',  # 土壤温度第一层
    'Q_2m': 'Q2',
    'T_2m': 'T2',
    'PSFC': 'PSFC',
    'Wind_10m': 'Wind10',
    'Prec': 'RAINNC',
}
obs_folder_mapping = {
    'ET': 'GLEAM',
    'LH': 'LH',
    'SH': 'GLEAM',
    'SSM': 'GLEAM',  # 土壤湿度第一层
    'RSM': 'GLEAM',  # 土壤湿度第一层
    'SST': 'LST',  # 土壤温度第一层
    'Q_2m': 'shum',
    'T_2m': 'temp',
    'PSFC': 'pres',
    'Wind_10m': 'wind',
    'Prec': 'CHM_PRE',
}
obs_var_mapping = {
    'ET': 'GLEAM_ET',
    'LH': 'LH',
    'SH': 'GLEAM_SH',
    'SSM': 'GLEAM_SSM',  # 土壤湿度第一层
    'RSM': 'GLEAM_RSM',  # 土壤湿度第一层
    'SST': 'LST',  # 土壤温度第一层
    'Q_2m': 'shum',
    'T_2m': 'temp',
    'PSFC': 'pres',
    'Wind_10m': 'wind',
    'Prec': 'Prec',
}

VarUnit = {
    'SSM': 'm³/m³',
    'ET': 'mm/day',
    'Q_2m': 'g/kg',
    'T_2m': 'K',
    'PSFC': 'hPa',
    'Wind_10m': 'm/s',
    'Prec': 'mm/day',
}
result_dict = {
    'Obs': {},
    'Sim': {},
            }
DataPath = '/raid61/yfdong/data/work/LF-SAM/output/evaluation/9km'



In [ ]:
import numpy as np
from netCDF4 import Dataset
def create_domain_mask(lat, lon, domain):
   domain_lat_min, domain_lat_max, domain_lon_min, domain_lon_max = domain
   mask = (lat >= domain_lat_min) & (lat <= domain_lat_max) & (lon >= domain_lon_min) & (lon <= domain_lon_max)
   masked_array = xr.DataArray(np.where(mask, 1, np.nan), dims=['south_north', 'west_east'])
   return masked_array
def extract_SubsetRegion_mask(var ,domain_mask):
    row_indices, col_indices = np.where(domain_mask == 1)  # 提取 ==1 的区域
    row_start, row_end = row_indices.min(), row_indices.max()
    col_start, col_end = col_indices.min(), col_indices.max()
    # 步骤2：切片获取子区域数据（保持二维结构）
    sub_var = var[row_start:row_end+1, col_start:col_end+1]
    return sub_var
# 文件路径
GeoPath = '/raid61/yfdong/data/work/LF-SAM/output/evaluation/9km/CHN_domain_9km.nc'
# 打开 NetCDF 文件
dst_nc = Dataset(GeoPath)
# 读取变量
lat_var = dst_nc.variables['lat'][:]
lon_var = dst_nc.variables['lon'][:]
mask = dst_nc.variables['Band1'][:]
lat_grid, lon_grid = np.meshgrid(lat_var, lon_var)
# 关闭 NetCDF 文件
dst_nc.close()
# 将 mask 数组转换为浮点类型
mask = mask.astype(float)
# 将 0 替换为 NaN
mask[mask == 0] = np.nan
print(lat_var.shape)
# 指定范围
Regions = ['ECN', 'SAM', 'JTB']
Years = [2011, 2018, 2020]
DomainRegion_ECN = [25, 35, 109, 122]
DomainRegion_SAM = [28.8, 30.8, 116.85, 120.0]
DomainRegion_JTB = [26.6, 28.4, 114.6, 117.4] 
Regions = ['SAM', 'JTB']
for RegionName in Regions:
    print(f"Processing RegionName: {RegionName}")
    if RegionName == 'ECN':
        DomainRegion = DomainRegion_ECN
    elif RegionName == 'SAM':
        DomainRegion = DomainRegion_SAM
    elif RegionName == 'JTB':
        DomainRegion = DomainRegion_JTB
    else:
        raise ValueError(f"Unknown RegionName: {RegionName}")
    lat_min, lat_max, lon_min, lon_max = DomainRegion
    # 找到指定范围内的索引
    lat_indices = np.where((lat_var >= lat_min) & (lat_var <= lat_max))[0]
    lon_indices = np.where((lon_var >= lon_min) & (lon_var <= lon_max))[0]
    # 创建一个与 mask 形状相同的全 NaN 数组
    mask_nan = np.full_like(mask, np.nan)
    # 将指定范围内的值复制到 mask_nan 中
    mask_nan[lat_indices.min():lat_indices.max()+1, lon_indices.min():lon_indices.max()+1] = mask[lat_indices.min():lat_indices.max()+1, lon_indices.min():lon_indices.max()+1]
    # plt.imshow(mask_nan)
    # 假设 num_times 是时间步数
    num_times = 1  # 请根据实际情况设置时间步数
    # 创建一个三维数组，形状为 (num_times, lat.shape[0], lon.shape[0])
    mask_3d = mask_nan[np.newaxis, :, :].repeat(1, axis=0)
    # -------------------------- 数据预处理模块 ------------------------------
    for year in Years:
        start_date = f'{year}-06-01 00:00:00'
        end_date = f'{year}-8-30 23:59:59'     
        # 生成时间序列，间隔为6小时
        dates = pd.date_range(start=start_date, end=end_date, freq='24H') 
        # 生成唯一的缓存文件名（包含关键参数）
        cache_filename = (
            f"LSMvar_evaluation_JJA_Daily_{RegionName}_{len(var_list)}vars"
            f"_{year}.nc"
        )
        print(year)
        cache_path = os.path.join(DataPath, 'cache', cache_filename)
        if os.path.exists(cache_path):
            print(f"缓存文件已存在: {cache_path}")
            # with nc.Dataset(cache_path, 'r') as ds:
            #     # 读取结果数据
            #     results = {
            #         'Ctrl': {var: ds[f'Ctrl_{var}'][:] for var in var_list},
            #         'Expl': {var: ds[f'Expl_{var}'][:] for var in var_list},
            #     }
        else:
            print(f"未找到缓存文件: {cache_path}，重新处理数据...")
            start = time.time()
            # 初始化结果字典
            result_dict = {'Obs': {}, 'WRF-S': {}, 'WRF-H': {}}
            for var in var_list:
                sim_folder = sim_folder_mapping[var]
                obs_folder = obs_folder_mapping[var]
                sim_var_name = sim_var_mapping[var]
                obs_var_name = obs_var_mapping[var]
                temp_WRF_S_var = np.zeros((len(dates), lat_var.shape[0], lon_var.shape[0]))
                temp_WRF_H_var = np.zeros((len(dates), lat_var.shape[0], lon_var.shape[0]))
                temp_obs_var = np.zeros((len(dates), lat_var.shape[0], lon_var.shape[0]))
                # print(sim_folder, obs_folder)
                for TimeIndex, date in enumerate(dates):
                    WRF_S_file = os.path.join(DataPath, 'sim', sim_folder, f'{date.strftime("%Y%m%d")}.WCTRL_{sim_folder}') 
                    WRF_H_file = os.path.join(DataPath, 'sim', sim_folder, f'{date.strftime("%Y%m%d")}.HCTRL_{sim_folder}') 
                    if var == 'Prec':
                        obs_file = os.path.join(DataPath, 'obs', obs_folder, 'output', f'{date.strftime("%Y%m%d")}.{obs_folder}') 
                    elif var in ['ET', 'SSM', 'RSM', 'SH']:
                        obs_file = os.path.join(DataPath, 'obs', obs_folder, 'output', f'{date.strftime("%Y%m%d")}00.{obs_var_name}')
                    else:
                        obs_file = os.path.join(DataPath, 'obs', obs_folder, 'output', f'{date.strftime("%Y%m%d")}00.{obs_folder}')
                    # 使用上下文管理器打开NetCDF文件
                    with nc.Dataset(WRF_S_file, 'r') as WRF_S_nc:
                        # 直接读取指定时间索引的数据，并写入目标数组
                        temp_WRF_S_var[TimeIndex, :, :] = WRF_S_nc.variables[sim_var_name][:] 
                    with nc.Dataset(WRF_H_file, 'r') as WRF_H_nc:
                        temp_WRF_H_var[TimeIndex, :, :] = WRF_H_nc.variables[sim_var_name][:] 
                    with nc.Dataset(obs_file, 'r') as obs_data:
                        # 同样地，直接读取并写入
                        temp_obs_var[TimeIndex, :, :] = obs_data.variables[obs_var_name][:] 
                        
                result_dict['WRF-S'][var] = temp_WRF_S_var *mask_3d
                result_dict['WRF-H'][var] = temp_WRF_H_var *mask_3d
                result_dict['Obs'][var] = temp_obs_var *mask_3d
            # ===================== 保存结果到缓存文件 =====================
            os.makedirs(os.path.dirname(cache_path), exist_ok=True)
            with nc.Dataset(cache_path, 'w') as ds:
                # 创建维度
                ds.createDimension('lat', lat_var.shape[0])
                ds.createDimension('lon', lon_var.shape[0])
                ds.createDimension('Date', result_dict['WRF-S'][var].shape[0])
                # 保存坐标变量
                lat = ds.createVariable('lat', 'f4', ('lat'))
                lon = ds.createVariable('lon', 'f4', ('lon'))
                Date = ds.createVariable('Date', 'f4', ('Date'))
                lat[:] = lat_var[:]
                lon[:] = lon_var[:]
                Date[:] = np.arange(0, result_dict['WRF-S'][var].shape[0])
                # 保存计算结果
                for var in var_list:
                    print(var)
                    WRF_S_var = ds.createVariable(f'WRF-S_{var}', 'f4', ('Date', 'lat', 'lon'), zlib =True)
                    WRF_H_var = ds.createVariable(f'WRF-H_{var}', 'f4', ('Date', 'lat', 'lon'), zlib =True)
                    obs_var = ds.createVariable(f'Obs_{var}', 'f4', ('Date', 'lat', 'lon'), zlib =True)
                    WRF_S_var[:] = result_dict['WRF-S'][var]
                    WRF_H_var[:] = result_dict['WRF-H'][var]
                    obs_var[:] = result_dict['Obs'][var]
                # 添加文件属性便于追溯
                ds.setncattr('Generated_Time', time.ctime())   


In [ ]:
# --------------------------------- 数据读取模块 ------------------------------------
# 定义年份列表
years = [2011, 2018, 2020]
# 初始化结果字典 - 三层结构：数据源 -> 年份 -> 变量
result_dict = {'Obs': {}, 'WRF-S': {}, 'WRF-H': {}}
# 初始化每个数据源的年字典
for source in ['Obs', 'WRF-S', 'WRF-H']:
    for year in years:
        result_dict[source][year] = {}
# 初始化每个变量的数据存储
for var in var_list:
    for year in years:
        result_dict['WRF-S'][year][var] = None
        result_dict['WRF-H'][year][var] = None
        result_dict['Obs'][year][var] = None
# ---------------------------------- 选择区域 ------------------------------
RegionName = 'SAM'
# --------------------------------------------------------------------------
# 处理每个年份
for YearIndex,year  in enumerate(years):
    start_date = f'{year}-06-01 00:00:00'
    end_date = f'{year}-08-30 23:59:59'
    
    # 生成时间序列，间隔为24小时
    dates = pd.date_range(start=start_date, end=end_date, freq='24H')
    
    # 构建缓存文件路径
    cache_filename = f"LSMvar_evaluation_JJA_Daily_{RegionName}_{len(var_list)}vars_{year}.nc"
    cache_path = os.path.join(DataPath, 'cache', cache_filename)
    
    # 打开 NetCDF 文件
    with Dataset(cache_path, 'r') as ds:
        for var in var_list:
            # 读取数据
            WRF_S_data = ds[f'WRF-S_{var}'][:]
            WRF_H_data = ds[f'WRF-H_{var}'][:]
            obs_data = ds[f'Obs_{var}'][:]
            # 质控
            WRF_S_data[WRF_S_data>10000]=np.nan
            WRF_H_data[WRF_H_data>10000]=np.nan
            obs_data[obs_data>10000]=np.nan
            if var == 'SSM':
                WRF_H_data[WRF_H_data>0.6]=np.nan
                WRF_S_data[WRF_S_data>0.6]=np.nan
                obs_data[obs_data>0.6]=np.nan
            # 将数据存储到对应年份
            result_dict['WRF-S'][year][var] = np.nanmean(WRF_S_data, axis=(1,2))
            result_dict['WRF-H'][year][var] = np.nanmean(WRF_H_data, axis=(1,2))
            result_dict['Obs'][year][var] = np.nanmean(obs_data, axis=(1,2))
    

# # 将结果字典中的列表转换为 numpy 数组
# for var in var_list:
#     result_dict['WRF-S'][var] = np.concatenate(result_dict['WRF-S'][var])
#     result_dict['WRF-H'][var] = np.concatenate(result_dict['WRF-H'][var])
#     result_dict['Obs'][var] = np.concatenate(result_dict['Obs'][var])

# 打印结果以验证
for var in var_list:
    print(f"Variable: {var}")
    print("Sim Data Shape:", result_dict['WRF-H'][year][var].shape)
    print("Obs Data Shape:", result_dict['Obs'][year][var].shape)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import MaxNLocator
# --------------------------------- 绘图模块 ------------------------------------
def PlotVar(Plot_result_dict , Year, axeIndex, VarName, ax1):
    # --------- 绘制土壤湿度模拟差异柱状图（左侧坐标轴）----------------------
    DateList = pd.date_range(start=f'{Year}-06-01', end=f'{Year}-08-30', freq='24H')
    # DiffVar = Plot_result_dict['WRF-H'][Year][VarName] - Plot_result_dict['WRF-S'][Year][VarName]
    # ax1.bar(DateList, (DiffVar), 
    #     width=1, alpha=1, 
    #     color="#009ACD", edgecolor='black', linewidth=0.5,
    #     label="Diff", zorder=1)
    # ax1.tick_params(axis='y', labelcolor="#104E8B")
    if VarName == 'Q_2m':
        PlotVarName = 'Q2m'
    elif VarName == 'T_2m':
        PlotVarName = 'T2m'
    else:
        PlotVarName = VarName
    ax1.set_ylim(VarRange[VarName][0], VarRange[VarName][1])
    ax1.yaxis.set_major_locator(MaxNLocator(nbins=5))
    ax1.set_ylabel(f"({VarUnit[VarName]})", fontsize=fontsize)
    LineWidth = 2
    # ----------- 绘制土壤湿度观测（右侧坐标轴）----------------------
    ax1.plot(DateList, Plot_result_dict['Obs'][Year][VarName], 
            color= "#363636", marker='o',markersize= 4, markeredgewidth=0.1,
            linewidth = LineWidth , alpha=0.8, label="OBS", zorder=10) #,zorder=10  
    # 绘制观测散点图
    # ax1.scatter(DateList, Plot_result_dict['Obs'][Year][VarName],
    #         color= "#363636", marker='o', s=20, edgecolors='black', linewidths=0.2, alpha=0.6,
    #         label="OBS", zorder=15) #,zorder=10
    ax1.plot(DateList, Plot_result_dict['WRF-H'][Year][VarName], 
            color= "#00BFFF", 
            linewidth = LineWidth , alpha=1, label="WRF-H", zorder=10) #,zorder=10  
    ax1.plot(DateList, Plot_result_dict['WRF-S'][Year][VarName], 
            color= "#FF7256", 
            linewidth = LineWidth , alpha=1, label="WRF-S", zorder=10) #,zorder=10  
    # 时间标签设置
    ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    ax1.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
    # 设置刻度标签字体大小
    ax1.tick_params(axis='x', labelsize=fontsize-1)  # 设置x轴刻度标签大小
    ax1.tick_params(axis='y', labelsize=fontsize-1)  # 设置y轴刻度标签大小
    # 调整刻度线参数：
    ax1.minorticks_on()
    # 显示副刻度线
    ax1.tick_params(axis="both", which="major", direction="in", width=0.9, length=5, color='#363636')
    # grid 设置网格线性
    ax1.grid(True, which="major", linestyle="--", color="gray", linewidth=0.75, alpha = 0.3)
    subTitle = f"({get_letter(axeIndex)}) {PlotVarName} ({Year})"
    ax1.set_title(subTitle, fontsize= fontsize, x= 0.03, y = 0.85, 
                  loc='left', weight='normal',  alpha=1)
    pass
VarRange = {
    'SSM': [0, 0.55],
    'ET': [0, 10],
    'Q_2m': [5, 25],
    'T_2m': [290, 310],
    'PSFC': [960, 990],
    'Wind_10m': [0, 6],
    'Prec': [0, 20],
}
DiffVarRange = {
    'SSM': [-0.2, 0.2],
    'ET': [-3, 3],
    'Q_2m': [-5, 5],    
    'T_2m': [-1, 5],
    'PSFC': [-0.5, 5],
    'Wind_10m': [-0.5, 5],
    'Prec': [-10, 10],
}
PlotVarList = ['SSM', 'ET', 'Q_2m']
PlotVarList = ['ET', 'Q_2m']
PlotVarList = ['T_2m', 'PSFC']
# PlotVarList = ['T_2m', 'PSFC', 'Wind_10m']
PlotYearList = [2011, 2018, 2020]
Rows = len(PlotVarList)
Cols = len(PlotYearList)
fontsize = 15
Plot_result_dict = result_dict.copy()
print("RegionName:", RegionName)
fig, axs = plt.subplots(Rows, Cols, figsize=(Cols*5/1.2, Rows*4/1.5), constrained_layout=True, dpi =300)
for IDx, Year in enumerate(PlotYearList):
    for  IDy,VarName in enumerate(PlotVarList):
        axeIndex = IDy * Cols + IDx
        axe = axs[IDy, IDx]
        PlotVar(Plot_result_dict, Year, axeIndex, VarName, axe)
    
# 创建统一的图例
lines, labels = fig.axes[-1].get_legend_handles_labels()
fig.legend( lines, labels,   
           fontsize= fontsize-0.5,        
            bbox_to_anchor=(0.68, 0.01),ncol=4, framealpha=1, edgecolor='none',
)
plt.suptitle(f"{RegionName} Region ", fontsize=fontsize, y=1.05)
# 设置子图间距
plt.subplots_adjust(wspace=0.1, hspace=0.1)  # 调整宽度和高度间距